In [3]:
import h5py, os
import numpy as np

In [4]:
f = h5py.File(os.getcwd()+'/data/lhe_data_gw.h5', 'r+')
#f = h5py.File(os.getcwd()+'/data/lhe_data_shuffled.h5', 'r+')

In [5]:
f.keys()

<KeysViewHDF5 ['benchmarks', 'morphing', 'observables', 'parameters', 'sample_summary', 'samples']>

In [6]:
bm = f['benchmarks']
bm.keys()

<KeysViewHDF5 ['is_nuisance', 'is_reference', 'names', 'values']>

In [7]:
mp = f['morphing']
mp.keys()

<KeysViewHDF5 ['components', 'morphing_matrix']>

In [8]:
ob = f['observables']
ob.keys()

<KeysViewHDF5 ['definitions', 'names']>

In [9]:
pm = f['parameters']
pm.keys()

<KeysViewHDF5 ['lha_blocks', 'lha_ids', 'max_power', 'names', 'ranges', 'transforms']>

In [10]:
ss = f['sample_summary']
ss.keys()

<KeysViewHDF5 ['background_events', 'signal_events_per_benchmark']>

In [11]:
samples = f['samples']
samples.keys()

<KeysViewHDF5 ['observations', 'sampling_benchmarks', 'weights']>

In [12]:
np.array(f['samples/sampling_benchmarks'])

array([3, 3, 4, ..., 3, 3, 3])

In [48]:
f.close()

### Samples

In [13]:
observations = np.array(samples['observations'])
observations.shape

(110000, 3)

In [14]:
weights = np.array(samples['weights'])
weights

array([[2.7841223e-08, 1.4210410e-08, 2.1442170e-08, ..., 2.0985015e-08,
        2.1245668e-08, 2.1771883e-08],
       [2.2302728e-08, 1.9083157e-08, 2.0893168e-08, ..., 2.0786713e-08,
        2.0847518e-08, 2.0969404e-08],
       [1.1378621e-07, 9.1049680e-08, 1.0373078e-07, ..., 1.0297749e-07,
        1.0340765e-07, 1.0427079e-07],
       ...,
       [2.0731574e-08, 2.0853072e-08, 2.0720938e-08, ..., 2.0724001e-08,
        2.0722183e-08, 2.0719090e-08],
       [2.0037817e-08, 2.1580510e-08, 2.0647189e-08, ..., 2.0697211e-08,
        2.0668568e-08, 2.0611724e-08],
       [2.9313322e-08, 1.3099329e-08, 2.1580261e-08, ..., 2.1034656e-08,
        2.1345611e-08, 2.1974413e-08]])

In [15]:
sampling_benchmarks = np.array(samples['sampling_benchmarks'])
print('num of sm events: '+str(sum(sampling_benchmarks == 3)))
print('num of 5 events: '+str(sum(sampling_benchmarks == 4)))
print('num of neg_5 events: '+str(sum(sampling_benchmarks == 5)))
print('num of 10 events: '+str(sum(sampling_benchmarks == 6)))
print('num of neg_10 events: '+str(sum(sampling_benchmarks == 7)))
print('num of 20 events: '+str(sum(sampling_benchmarks == 8)))
print('num of neg_20 events: '+str(sum(sampling_benchmarks == 9)))

num of sm events: 50000
num of 5 events: 10000
num of neg_5 events: 10000
num of 10 events: 10000
num of neg_10 events: 10000
num of 20 events: 10000
num of neg_20 events: 10000


### Embedding toy data

In [16]:
toydataFile = h5py.File(os.getcwd()+'/toydata/gw_toydata.h5', 'r')

#### data

In [17]:
del f['samples/observations']
f.create_dataset('samples/observations', data=np.array(toydataFile['Data']))

<HDF5 dataset "observations": shape (6000000, 9), type "<f4">

#### sampling_benchmarks

In [18]:
N = 500000
sampling_benchmarks = np.concatenate([np.concatenate([np.ones(N)*3, np.ones(N)*i]) for i in range(4, 10)])
del f['samples/sampling_benchmarks']
f.create_dataset('samples/sampling_benchmarks', data=sampling_benchmarks)

<HDF5 dataset "sampling_benchmarks": shape (6000000,), type "<f8">

#### weights

In [19]:
Weights = np.array(toydataFile['Weights'])

In [20]:
Weights

array([1.2393269e-07, 1.2393269e-07, 1.2393269e-07, ..., 3.1926135e-07,
       3.1926135e-07, 3.1926135e-07], dtype=float32)

In [21]:
weights = np.ones([N*12, 10])*(1e10)

In [20]:
#weights[N*0:N*1, 3] = Weights[N*0]
#weights[N*1:N*2, 4] = Weights[N*1]
#weights[N*:N*1, 3] = Weights[N*0]

In [22]:
for i in range(12):
    if i%2==0:
        print('sm data, pos=%d'%(3))
        weights[N*i:N*(i+1), 3] = Weights[N*i]
    else:
        print('bsm data, pos=%d'%(3+(i+1)/2))
        weights[N*i:N*(i+1), int(3+(i+1)/2)] = Weights[N*i]

sm data, pos=3
bsm data, pos=4
sm data, pos=3
bsm data, pos=5
sm data, pos=3
bsm data, pos=6
sm data, pos=3
bsm data, pos=7
sm data, pos=3
bsm data, pos=8
sm data, pos=3
bsm data, pos=9


In [23]:
del f['samples/weights']
f.create_dataset('samples/weights', data=weights)

<HDF5 dataset "weights": shape (6000000, 10), type "<f8">

#### observable numbers

In [24]:
f['observables'].keys()

<KeysViewHDF5 ['definitions', 'names']>

In [25]:
print(np.array(f['observables/definitions']))
print(np.array(f['observables/names']))

[b'j[0].pt' b'j[0].deltaphi(j[1]) * (-1. + 2.*float(j[0].eta > j[1].eta))'
 b'met.pt']
[b'pt_j1' b'delta_phi_jj' b'met']


In [26]:
del f['observables/definitions']
del f['observables/names']

observables = ['s', 'theta', 'thetaZ', 'thetaW', 'Sin(phiZ)', 'Sin(phiW)', 'Cos(phiZ)', 'Cos(phiW)', 'Pt']
f.create_dataset('observables/definitions', data=(np.array(observables, dtype='S')))
f.create_dataset('observables/names', data=(np.array(observables, dtype='S')))

<HDF5 dataset "names": shape (9,), type "|S9">

In [27]:
f.close()